In [2]:
# download reward model
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

rm_tokenizer = AutoTokenizer.from_pretrained("usvsnsp/pythia-6.9b-rm-full-hh-rlhf")
rm = AutoModelForSequenceClassification.from_pretrained("usvsnsp/pythia-6.9b-rm-full-hh-rlhf")
seq_length = 1024
if torch.cuda.is_available():
    rm = rm.cuda()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Try on a single example

In [3]:
prompt = """\n\nHuman: What is the modern day concept of a "soul mate"?\n\nAssistant:"""
selected = """ Ah, soul mates! A lot of romantic relationships depend on the concept of a soul mate, so you can consider soul mates the ultimate expression of romantic love. Soul mates are supposed to be a relationship of profound attraction and deep connection, a meeting of two "soul twins" who are almost one."""
#prompt_converted, selected_converted, _ = to_vicuna_format({"prompt": prompt, "selected": selected, "rejected": ""}).values()
tokenized = rm_tokenizer(prompt + selected + rm_tokenizer.eos_token, truncation=True, max_length=seq_length, return_tensors="pt")
if torch.cuda.is_available():
    tokenized = tokenized.to("cuda")
rm_output = rm(**tokenized) 
rm_output[0]

tensor([[-2.1031]], device='cuda:0', grad_fn=<IndexBackward0>)

# Compute accuracy

In [5]:
from datasets import load_dataset

#dataset = load_dataset("Anthropic/hh-rlhf")
dataset = load_dataset("pvduy/rm_oa_hh")

Found cached dataset parquet (/scratch/lgrinszt/pvduy___parquet/pvduy--rm_oa_hh-b0e03bd708a894fe/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
import torch
rm_tokenizer.add_special_tokens({"pad_token": "<|padding|>"})
rm_tokenizer.padding_side = "right"
rm_tokenizer.truncation_side = "left"
seq_length=1024

def tokenize(prompt, selected, rejected, tokenizer):
    return {
        "selected_input_ids": tokenizer(prompt + selected + tokenizer.eos_token, truncation=True, max_length=seq_length).input_ids,
        "rejected_input_ids": tokenizer(prompt + rejected + tokenizer.eos_token, truncation=True, max_length=seq_length).input_ids,
    }

def collate_fn(batch):
    input_ids = sum([[x["rejected_input_ids"], x["selected_input_ids"]] for x in batch], [])
    return rm_tokenizer.pad({"input_ids": input_ids}, padding=True, return_tensors="pt")

if "chosen" in dataset["train"].column_names:
    dataset = dataset.rename_column("chosen", "selected")
if "replies" in dataset["train"].column_names:
    dataset = dataset.map(lambda x: {"selected": x["replies"][0], "rejected": x["replies"][1]}, remove_columns=["replies"])

def to_vicuna_format(sample):
    prompt = sample["prompt"].strip()
    prompt = prompt.replace("\n\nHuman: ", "</s>USER: ") \
                    .replace("\n\nAssistant: ", " ASSISTANT: ") \
                    .replace("\n\nAssistant:", " ASSISTANT:")
    if prompt.startswith("Human: "):
        prompt = prompt.replace("Human: ", "USER: ")
    if prompt.startswith("</s>"):
        prompt = prompt[4:]

    selected = " " + sample["selected"].strip()
    rejected = " " + sample["rejected"].strip()

    return {"prompt": prompt, "selected": selected, "rejected": rejected}

def to_oa_format(sample):
    prompt = sample["prompt"].strip()
    prompt = prompt.replace("\n\nHuman: ", "</s><|prompter|>") \
                    .replace("\n\nAssistant: ", "</s><|assistant|>") \
                    .replace("\n\nAssistant:", "</s><|assistant|>")
    if prompt.startswith("Human: "):
        prompt = prompt.replace("Human: ", "<|prompter|>")

    selected = sample["selected"].strip()
    rejected = sample["rejected"].strip()

    return {"prompt": prompt, "selected": selected, "rejected": rejected}

# if args.add_oasst_tokens:
#dataset = dataset.map(to_oa_format)
# else:
#dataset = dataset.map(to_vicuna_format)


tokenized = dataset.map(tokenize, input_columns=["prompt", "selected", "rejected"], fn_kwargs=dict(tokenizer=rm_tokenizer), desc="Tokenizing")
#dataloader = torch.utils.data.DataLoader(tokenized["train"], shuffle=True, batch_size=32, collate_fn=collate_fn)

Tokenizing:   0%|          | 0/166750 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/8524 [00:00<?, ? examples/s]

In [11]:
eval_dataloader = torch.utils.data.DataLoader(tokenized["test"], shuffle=True, batch_size=2, collate_fn=collate_fn)

In [12]:
all_scores, all_delta_scores, all_tokens = [], [], []
# move dataloader to GPU
import numpy as np
rm.eval()
rm = rm.to("cuda")
for batch in eval_dataloader:
        # move batch to GPU
        if torch.cuda.is_available():
            batch = {k: v.to("cuda") for k, v in batch.items()}
        with torch.no_grad():
            scores = rm(**batch)[0]

        delta_scores = scores.reshape(-1, 2).diff().view(-1)
        all_delta_scores.extend(delta_scores.tolist())
        all_scores.extend(scores.view(-1).tolist())
        all_tokens.extend(batch["input_ids"].tolist())

        delta_scores = np.hstack(all_delta_scores)
        accuracy = (delta_scores > 0).mean()
        print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.00
Accuracy: 0.25
Accuracy: 0.50
Accuracy: 0.50
Accuracy: 0.50
Accuracy: 0.50
Accuracy: 0.50
Accuracy: 0.50
Accuracy: 0.44
Accuracy: 0.45
Accuracy: 0.41
Accuracy: 0.42
Accuracy: 0.42
Accuracy: 0.43
Accuracy: 0.47
Accuracy: 0.50
Accuracy: 0.50
Accuracy: 0.47
Accuracy: 0.47
Accuracy: 0.50
Accuracy: 0.52
Accuracy: 0.55
Accuracy: 0.57
Accuracy: 0.56
Accuracy: 0.58
Accuracy: 0.56
Accuracy: 0.56
Accuracy: 0.55
Accuracy: 0.53
Accuracy: 0.53
Accuracy: 0.53
Accuracy: 0.53
Accuracy: 0.53
Accuracy: 0.51
Accuracy: 0.51
Accuracy: 0.50
Accuracy: 0.50
Accuracy: 0.50
Accuracy: 0.51
Accuracy: 0.50
Accuracy: 0.51
Accuracy: 0.52
Accuracy: 0.53
Accuracy: 0.53
Accuracy: 0.53
Accuracy: 0.52
Accuracy: 0.52
Accuracy: 0.52
Accuracy: 0.51
Accuracy: 0.51
Accuracy: 0.50
Accuracy: 0.49
Accuracy: 0.50
Accuracy: 0.50
Accuracy: 0.51
Accuracy: 0.51
Accuracy: 0.51
Accuracy: 0.52
Accuracy: 0.53
Accuracy: 0.53
Accuracy: 0.53
Accuracy: 0.53
Accuracy: 0.52
Accuracy: 0.52
Accuracy: 0.52
Accuracy: 0.51
Accuracy: 

KeyboardInterrupt: 